In [1]:
### Imports ###
import mrcfile
import torch
import numpy as np
import matplotlib.pyplot as plt
from nntomo.utilities import get_MSE_loss
from nntomo.network import model_training, get_model_infos, edit_model
from nntomo.dataset_slices import DatasetSlices
from nntomo.projection_stack import ProjectionStack
from nntomo.volume import Volume

%load_ext autoreload
%autoreload 2

### Training dataset

For the training dataset, simulated STEM projections fo different MOF structures are taken for the inputs and the SIRT reconstruction with 72 projections is used for the real output values. The selected MOF structures are: MOF-5, PCN-61, IRMOF-76, ZIF-4 and MOF-1131.

In [2]:
training_mofs = ['MOF-5', 'PCN-61', 'IRMOF-76', 'ZIF-4', 'MOF-1131']
cell_repetitions = [3, 2, 2, 5, 3] # Chosen so that the number of voxels around the first axis is roughly the same
symetries = [True, True, True, False, False]

training_stacks = []

for mof, cell_repetition, symetry in zip(training_mofs, cell_repetitions, symetries):
    cif_file = f"data/cif_files/{mof}.cif"
    projections = ProjectionStack.from_cif_file(cif_file, 72, 'full', cell_repetition=(cell_repetition,1,1), del_hydrogen=True, symetry=symetry)
    projections = projections.get_resized_proj(512)
    projections.save()
    training_stacks.append(projections)

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'cubic' is not interpreted for space group Spacegroup(225, setting=1). This may result in wrong setting!
  warnings.warn(


tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'Cubic' is not interpreted for space group Spacegroup(225, setting=1). This may result in wrong setting!
  warnings.warn(


tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'Orthorhombic' is not interpreted for space group Spacegroup(61, setting=1). This may result in wrong setting!
  warnings.warn(


tif file retrieved for projection angle -90.0°
tif file retrieved for projection angle -87.5°
tif file retrieved for projection angle -85.0°
tif file retrieved for projection angle -82.5°
tif file retrieved for projection angle -80.0°
tif file retrieved for projection angle -77.5°
tif file retrieved for projection angle -75.0°
tif file retrieved for projection angle -72.5°
tif file retrieved for projection angle -70.0°
tif file retrieved for projection angle -67.5°
tif file retrieved for projection angle -65.0°
tif file retrieved for projection angle -62.5°
tif file retrieved for projection angle -60.0°
tif file retrieved for projection angle -57.5°
tif file retrieved for projection angle -55.0°
tif file retrieved for projection angle -52.5°
tif file retrieved for projection angle -50.0°
tif file retrieved for projection angle -47.5°
tif file retrieved for projection angle -45.0°
tif file retrieved for projection angle -42.5°
tif file retrieved for projection angle -40.0°
tif file retr

KeyboardInterrupt: 

In [38]:
sirt_reconstructions = [stack.get_SIRT_reconstruction(150) for stack in training_stacks]


Start of SIRT reconstruction.
Saving volume...
File saved at data/volume_files/sirt150_srtio3_100_20204_stem_72proj_resized512-512.mrc. ID: sirt150_srtio3_100_20204_stem_72proj_resized512-512


<pre>
The 72 projections:                The SIRT reconstruction (isosurfarce value: 138/255):

<img src="data/gifs/srtio320204_projs.gif" width="300"/>            <img src="data/gifs/srtio320204_sirt72.gif" width="300"/>
</pre>

In [50]:
training_dataset = DatasetSlices(srtio3_72projections.get_proj_subset(9), srtio3_sirt_reconstruction.get_segmented_volume(138/255))
training_dataset.save()
srtio3_72projections.get_proj_subset(9).save()

Saving dataset...
File saved at data/datasets_files/srtio3_100_20204_stem_72proj_resized512-512_sub9_sirt150_srtio3_100_20204_stem_72proj_resized512-512_segm0.54_bin.pickle. ID: srtio3_100_20204_stem_72proj_resized512-512_sub9_sirt150_srtio3_100_20204_stem_72proj_resized512-512_segm0.54_bin
Saving projections...
File saved at data/projection_files/srtio3_100_20204_stem_72proj_resized512-512_sub9.mrc. ID: srtio3_100_20204_stem_72proj_resized512-512_sub9


### MOF

In [33]:
### Structure for NN training - MOF: projections ###
mof_cif_file = "data/cif_files/1021002.cif"
mof_projections = ProjectionStack.from_cif_file(mof_cif_file, 72, 'full', cell_repetition=(3,3,1), mode='stem', detector_type='haadf', del_hydrogen=True)
mof_projections.get_resized_proj((512,512)).save()

c:\Users\Admin-tomo\anaconda3\envs\abtem_env\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'cubic' is not interpreted for space group Spacegroup(225, setting=1). This may result in wrong setting!
  warnings.warn(


file retrieved for projection angle -90.0°
file retrieved for projection angle -87.5°
file retrieved for projection angle -85.0°
file retrieved for projection angle -82.5°
file retrieved for projection angle -80.0°
file retrieved for projection angle -77.5°
file retrieved for projection angle -75.0°
file retrieved for projection angle -72.5°
file retrieved for projection angle -70.0°
file retrieved for projection angle -67.5°
file retrieved for projection angle -65.0°
file retrieved for projection angle -62.5°
file retrieved for projection angle -60.0°
file retrieved for projection angle -57.5°
file retrieved for projection angle -55.0°
file retrieved for projection angle -52.5°
file retrieved for projection angle -50.0°
file retrieved for projection angle -47.5°
file retrieved for projection angle -45.0°
file retrieved for projection angle -42.5°
file retrieved for projection angle -40.0°
file retrieved for projection angle -37.5°
file retrieved for projection angle -35.0°
file retrie

In [40]:
### Structure for NN training - MOF: reference volume (SIRT reconstruction with 72 projections) ###
mof_72projections = ProjectionStack.retrieve("1021002_331_stem_72proj_resized512-512", 'full')
mof_sirt_reconstruction = mof_72projections.get_SIRT_reconstruction(150)
mof_sirt_reconstruction.save()

Start of SIRT reconstruction.
Saving volume...
File saved at data/volume_files/sirt150_1021002_331_stem_72proj_resized512-512.mrc. ID: sirt150_1021002_331_stem_72proj_resized512-512


<pre>
The 72 projections:                The SIRT reconstruction (isosurfarce value: 128/255):

<img src="data/gifs/mof331_projs.gif" width="300"/>            <img src="data/gifs/mof331_sirt72.gif" width="300"/>
</pre>

In [41]:
validation_dataset = DatasetSlices(mof_72projections.get_proj_subset(9), mof_sirt_reconstruction.get_segmented_volume(128/255))
validation_dataset.save()

Saving dataset...
File saved at data/datasets_files/1021002_331_stem_72proj_resized512-512_sub9_sirt150_1021002_331_stem_72proj_resized512-512_segm0.50_bin.pickle. ID: 1021002_331_stem_72proj_resized512-512_sub9_sirt150_1021002_331_stem_72proj_resized512-512_segm0.50_bin


### NN training

In [51]:
#reverse train and valid
validation_dataset = DatasetSlices.retrieve("srtio3_100_20204_stem_72proj_resized512-512_sub9_sirt150_srtio3_100_20204_stem_72proj_resized512-512_segm0.54_bin")
training_dataset = DatasetSlices.retrieve("1021002_331_stem_72proj_resized512-512_sub9_sirt150_1021002_331_stem_72proj_resized512-512_segm0.50_bin")

network = model_training(training_dataset, validation_dataset, 8, max_epoch=50)

Start of training.
Epoch 1 (n=0)
-------------------------------
Avg MSELoss(): 0.033562 

Epoch 2 (n=0)
-------------------------------
Avg MSELoss(): 0.012280 

Epoch 3 (n=0)
-------------------------------
Avg MSELoss(): 0.008127 

Epoch 4 (n=0)
-------------------------------
Avg MSELoss(): 0.007408 

Epoch 5 (n=0)
-------------------------------
Avg MSELoss(): 0.007267 

Data saved.
Epoch 6 (n=0)
-------------------------------
Avg MSELoss(): 0.007243 

Epoch 7 (n=0)
-------------------------------
Avg MSELoss(): 0.007240 

Epoch 8 (n=0)
-------------------------------
Avg MSELoss(): 0.007240 

Epoch 9 (n=0)
-------------------------------
Avg MSELoss(): 0.007240 

Epoch 10 (n=0)
-------------------------------
Avg MSELoss(): 0.007240 

Data saved.
Epoch 11 (n=1)
-------------------------------
Avg MSELoss(): 0.007240 

Epoch 12 (n=2)
-------------------------------
Avg MSELoss(): 0.007240 

Epoch 13 (n=3)
-------------------------------
Avg MSELoss(): 0.007239 

Epoch 14 (n=0)
--

### Reconstructions

In [57]:
mof_9projections = srtio3_72projections.get_proj_subset(9)
nn_reconstruction = mof_9projections.get_NN_reconstruction(network)
nn_reconstruction.save()

Start of NN reconstruction.
Reconstruction part 1/2: [████████████████████████████████████████████████████████████] 8/8 Est wait 00:0.0.0

Reconstruction part 2/2: [████████████████████████████████████████████████████████████] 8/8 Est wait 00:0.0.0

Saving volume...
File saved at data/volume_files/nn_1021002_331_stem_72proj_resized512-512_sub9_sirt150_1021002_331_stem_72proj_resized512-512_segm0.50_bin_8h_srtio3_100_20204_stem_72proj_resized512-512_sub9.mrc. ID: nn_1021002_331_stem_72proj_resized512-512_sub9_sirt150_1021002_331_stem_72proj_resized512-512_segm0.50_bin_8h_srtio3_100_20204_stem_72proj_resized512-512_sub9


In [47]:
sirt9_reconstruction = mof_9projections.get_SIRT_reconstruction(150)
sirt9_reconstruction.save()

Start of SIRT reconstruction.
Saving volume...
File saved at data/volume_files/sirt150_1021002_331_stem_72proj_resized512-512_sub9.mrc. ID: sirt150_1021002_331_stem_72proj_resized512-512_sub9
